In [1]:
# perform lemmatization (not recommended when using any pretrained embedding)
LEMMA = False
# size of the vocab for
VOCAB_SIZE = 20000
# for self attention
DATA_POINTS = 10
DIMENSIONS = 240
MAX_SEQUENCE_LENGTH = 256
# use gpu
USE_GPU = 0

In [2]:
# !pip install tensorflow

In [3]:
# !pip install keras

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14436410271488284987
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7806648320
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14561076554466654689
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:07:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [5]:
# !python -m spacy download en_core_web_md

In [29]:
from tensorflow import keras
import tensorflow as tf
from keras.layers import Input, Dense, LSTM, Flatten, concatenate, Activation, RepeatVector, Permute
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
#from keras_self_attention import SeqSelfAttention
from random import randint
from numpy import array, argmax, asarray, zeros
import numpy as np 
import pandas as pd
import spacy
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from textacy import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
nlp = spacy.load('en_core_web_md')

In [7]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
df = pd.read_csv('complaints_processed.csv')
df.head()


,Unnamed: 0,product,narrative
0,0,credit_card,purchase order day shipping amount receive pro...
1,1,credit_card,forwarded message date tue subject please inve...
2,2,retail_banking,forwarded message cc sent friday pdt subject f...
3,3,credit_reporting,payment history missing credit report speciali...
4,4,credit_reporting,payment history missing credit report made mis...


In [9]:
complaint_types = list(df['product'].value_counts().index)

In [10]:
NUM_CLASSES = len(complaint_types)

In [11]:
df['narrative'] = df['narrative'].apply(str)

In [12]:
preproc = preprocessing.make_pipeline(
    preprocessing.normalize.hyphenated_words,
    preprocessing.remove.accents,
)

In [13]:
set(stopwords.words('english'))
# removing some negative words from stopwords list
nltk_stopwords = set(stopwords.words('english'))
nltk_stopwords.remove('below')
nltk_stopwords.remove("aren't")
nltk_stopwords.remove('couldn')
nltk_stopwords.remove("couldn't")
nltk_stopwords.remove("didn't")
nltk_stopwords = list(nltk_stopwords)
# add some abstract terms
nltk_stopwords.append('like')
nltk_stopwords.append('please')

nltk_stopwords = set(nltk_stopwords)

In [14]:
# removing all words with more than 15 digits
df['narrative'] = df['narrative'].str.replace(r'\b([a-z]|[A-Z]){15,}\b', '', case=False)

C:\Users\Robert\AppData\Local\Temp\ipykernel_2148\868481445.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['narrative'] = df['narrative'].str.replace(r'\b([a-z]|[A-Z]){15,}\b', '', case=False)


In [15]:
# removing all characters that appear more than 2 times
df['narrative'] = df['narrative'].str.replace(r'(.)\1{3,}?', '', case=False)


C:\Users\Robert\AppData\Local\Temp\ipykernel_2148\1200782744.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['narrative'] = df['narrative'].str.replace(r'(.)\1{3,}?', '', case=False)


In [16]:
labels = df['product']

2. model preparation
    1. tokenize
    2. integer encoding
    

In [17]:
# tokenize text
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(df['narrative'])

In [18]:
# integer encoding
from typing import List
def get_max_token_length_per_doc(docs: List[List[str]])-> int:
    return max(list(map(lambda x: len(x.split()), docs)))

# get the max length in terms of token length
max_length = get_max_token_length_per_doc(df['narrative'])


def integer_encode_documents(docs, tokenizer):
    return tokenizer.texts_to_sequences(docs)

from keras.preprocessing.sequence import pad_sequences
# integer encode the documents
encoded_docs = integer_encode_documents(df['narrative'], tokenizer)
# this is a list of lists, the numbers represent the index position of that word.
# for instance, 33 means the 33rd word in the vocabulary
padded_docs = pad_sequences(encoded_docs, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [19]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
transfomed_labels = encoder.fit_transform(labels)
# print(transfomed_labels)

In [20]:
# %pip install skmultilearn

In [21]:
from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import iterative_train_test_split
# we did not use stratifty due to multiclass complications
X_train, X_test, y_train, y_test = iterative_train_test_split(padded_docs, transfomed_labels, test_size=0.2)

3. Bert
- Modified after: https://colab.research.google.com/github/CyberZHG/keras-bert/blob/master/demo/tune/keras_bert_classification_tpu.ipynb

In [22]:
BATCH_SIZE = 128
EPOCHS = 5
LR = 1e-4

In [23]:
%pip install -q keras-bert keras-rectified-adam

Note: you may need to restart the kernel to use updated packages.


In [24]:
import os

pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

# TF_KERAS must be added to environment variables in order to use TPU
os.environ['TF_KERAS'] = '1'

In [25]:
import codecs
from keras_bert import load_trained_model_from_checkpoint

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)


model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=MAX_SEQUENCE_LENGTH,
)

In [30]:
from tensorflow.python import keras
# from keras_radam import RAdam
from tensorflow.keras.optimizers import Adam


inputs = model.inputs[:2]
dense = model.get_layer('NSP-Dense').output
outputs = keras.layers.Dense(units=2, activation='softmax')(dense)

model = keras.models.Model(inputs, outputs)
model.compile(loss="categorical_crossentropy", optimizer= "adam", metrics=['accuracy'])

TypeError: Cannot convert a symbolic Keras input/output to a numpy array. This error may indicate that you're trying to pass a symbolic value to a NumPy call, which is not supported. Or, you may be trying to pass Keras symbolic inputs/outputs to a TF API that does not register dispatching, preventing Keras from automatically converting the API call to a lambda layer in the Functional Model.

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K

sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

In [ ]:
model.fit(
    X_train,
    train_y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
)